## Урок 1. Алгоритм линейной регрессии. Градиентный спуск

**Задача 1**

Проведите небольшое исследование алгоритма градиентного спуска. Оцените влияние значений скорости обучения (eta) и количества итераций на ошибку алгоритма. Как связаны эти два гиперпараметра между собой? Подберите скорость обучения и количество итераций до совпадения ответов алгоритма с результатами МНК. Как можно ускорить процесс вычисления весов?

In [196]:
import numpy as np

__Задача:__ предсказание баллов ЕГЭ ученика в зависимости от количества лет стажа его репетитора

In [197]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2]])
X

array([[ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [ 1,  1,  2,  5,  3,  0,  5, 10,  1,  2]])

In [198]:
X.shape

(2, 10)

In [199]:
y = [45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

In [200]:
#назодим правильные веса и с помощью метода наименьих квадратов (аналитическое решение)
W = np.linalg.inv(np.dot(X, X.T)) @ X @ y
W

array([45.0625,  3.8125])

In [201]:
X[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [202]:
#находим предсказание по y
y_pred = W[0] * X[0] + W[1] * X[1]
y_pred

array([48.875 , 48.875 , 52.6875, 64.125 , 56.5   , 45.0625, 64.125 ,
       83.1875, 48.875 , 52.6875])

In [203]:
#функции для вычисления ошибок
def calc_mae(y, y_pred):
    err = np.mean(np.abs(y - y_pred))
    return err

def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2) # <=> 1/n * np.sum((y_pred - y)**2)
    return err

In [204]:
calc_mae(y, y_pred), calc_mse(y, y_pred)

(5.7875, 43.96874999999999)

Градиентный спуск

In [205]:
n = 10
Q = 1/n * np.sum((y_pred - y)**2) # функционал ошибки, y = X*w

In [206]:
Q

43.96875

In [207]:
alpha = 1e-2 # величина шага
#градиент по первому весу
g = alpha * (1/n * 2 * np.sum(X[0] * (W[0] * X[0] - y)))
g

-0.22874999999999973

Алгоритм градиентного спуска

In [208]:
n = X.shape[1]
alpha = 1e-2
W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {alpha} \
       \nInitial weights = {W} \n')

for i in range(100):
    y_pred = np.dot(W, X)
    err = calc_mse(y, y_pred)
    for k in range(W.shape[0]):
        W[k] -= alpha * (1/n * 2 * np.sum(X[k] * (y_pred - y)))
    if i % 10 == 0:
        alpha /= 1.1
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')

Number of objects = 10        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [2.08 4.27], MSE = 3047.75
Iteration #10: W_new = [ 6.67106886 10.61676385], MSE = 749.71
Iteration #20: W_new = [ 9.49320908 10.25731657], MSE = 648.91
Iteration #30: W_new = [11.85740092  9.83349244], MSE = 570.46
Iteration #40: W_new = [13.86876921  9.46898661], MSE = 508.03
Iteration #50: W_new = [15.59085668  9.15672679], MSE = 457.73
Iteration #60: W_new = [17.07337653  8.88789585], MSE = 416.77
Iteration #70: W_new = [18.35601294  8.65530964], MSE = 383.06
Iteration #80: W_new = [19.47073522  8.45317196], MSE = 355.08
Iteration #90: W_new = [20.44350656  8.27677488], MSE = 331.65


Подберем скорость обучения и количество итераций до совпадения ответов алгоритма с результатами МНК.

MSE = 43.96874999999999

W = array([45.0625,  3.8125])

In [209]:
#реализуем градиентный спуск в виде функции
def gradient_decent(alpha, iterations, X=X, y=y):
    n = X.shape[1]
    W = np.array([1, 0.5])
    for i in range(iterations):
        y_pred = np.dot(W, X)
        err = calc_mse(y, y_pred)
        for k in range(W.shape[0]):
            W[k] -= alpha * (1/n * 2 * np.sum(X[k] * (y_pred - y)))
        if i % 100 == 0:
            alpha /= 1.1
    return i, W, err

In [210]:
a = gradient_decent(1e-2, 100)

In [211]:
print(a)

(99, array([26.77617698,  7.12844274]), 204.10741643337548)


In [212]:
#теперь найдем оптимальные альфа и количество итераций
alpha_variants = [1e-2, 2e-2, 1e-3]
iter_variants = [1e2, 2e2, 1e3, 1001, 2e3, 1e4, 1e5, 1e6] 

def best_gd_params(alpha_variants, iter_variants):
        W_best = []
        err_min = np.inf
        iter_num = 0
        alpha_num = 0
        for i in alpha_variants:
            for j in iter_variants:
                iter_new, W, err = gradient_decent(i, int(j))
                print(f'Альфа = {i}, количество итераций = {j}. Ошибка: {err}. Веса {W}')
                if err < err_min:
                    err_min = err
                    W_best = W
                    alpha_num  = i
                    iter_num = iter_new
        return err_min, W_best, alpha_num, iter_num

In [213]:
err_min, W_best, alpha_num, iter_num = best_gd_params(alpha_variants, iter_variants)

Альфа = 0.01, количество итераций = 100.0. Ошибка: 204.10741643337548. Веса [26.77617698  7.12844274]
Альфа = 0.01, количество итераций = 200.0. Ошибка: 79.12723669428905. Веса [36.48771351  5.36740532]
Альфа = 0.01, количество итераций = 1000.0. Ошибка: 43.979643880445984. Веса [44.91095014  3.83998123]
Альфа = 0.01, количество итераций = 1001. Ошибка: 43.979567406617754. Веса [44.91148301  3.8398846 ]
Альфа = 0.01, количество итераций = 2000.0. Ошибка: 43.96889277338202. Веса [45.04511285  3.81565289]
Альфа = 0.01, количество итераций = 10000.0. Ошибка: 43.968759440846135. Веса [45.05802288  3.81331186]
Альфа = 0.01, количество итераций = 100000.0. Ошибка: 43.968759428333655. Веса [45.05802584  3.81331132]
Альфа = 0.01, количество итераций = 1000000.0. Ошибка: 43.968759428333655. Веса [45.05802584  3.81331132]
Альфа = 0.02, количество итераций = 100.0. Ошибка: 74.29668302268078. Веса [37.17110724  5.24348241]
Альфа = 0.02, количество итераций = 200.0. Ошибка: 45.41378989581208. Веса 

In [214]:
print(f'Лучшие параметры для градиентного спуска: alpha = {alpha_num}, iterations = {iter_num}')
print(f'Коэффициенты: {W_best}, MSE = {err_min}')

Лучшие параметры для градиентного спуска: alpha = 0.02, iterations = 99999
Коэффициенты: [45.06249954  3.81250008], MSE = 43.96875000000009


**Ответы по задаче 1**

1. Оцените влияние значений скорости обучения (eta) и количества итераций на ошибку алгоритма. Как связаны эти два гиперпараметра между собой?

При увеличении скорости обучения - оптимальную ошибку алгоритма можно найти быстрее, но есть риск "перескочить" оптимальную ошибку при слишком большой шаге обучения и тогда ошибка вообще не будет найдена. Количество итераций влияет прямо пропоурционально на поиск ошибки алгоритма - чем больше итераций, тем быстрее мы приближаемся к оптимальной ошибке (если конечно мы ее не пропустили из-за слишком большого шага обучения).
Эти два параметра связаны с собой обратно пропорционально, чем больше скорость обучения, тем меньше нужно итераций. Чем больше итераций, тем меньше потребуется скорость обучения.

 2. Подберите скорость обучения и количество итераций до совпадения ответов алгоритма с результатами МНК.
 
 Решение выше. Параметры получились:
 
Лучшие параметры для градиентного спуска: alpha = 0.02, iterations = 99999

Коэффициенты: [45.06249954  3.81250008], MSE = 43.96875000000009
 
 В целом корректное значение ошибки с округлением до 5го знака после запятой удалось обнаружить уже при параметрах:
 
 Альфа = 0.01, количество итераций = 10000.0. Ошибка: 43.968759440846135. Веса [45.05802288  3.81331186]

3. Как можно ускорить процесс вычисления весов?

Можно путем увеличения скорости обучений, и постепенно уменьшении ее, при приближении к ошибке (то есть сначала мы "шагаем" большими шагами, не боимся пропустить ошибку, потом постепенно уменьшаем шаги). 

**Задача 2**

In [215]:
#(*) В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее:

# w = np.array([1, 0.5])
# for i in range(1001):
#     y_pred = np.dot(w, X.T)
#     err = calc_mse(y, y_pred)
#     w -= (eta * (1/n * 2 * np.sum(X.T * (y_pred - y)))) # ошибка!
#     if i % 100 == 0:
#         print(i, w, err)

In [216]:
#опишу изменения
#в результате градиентный спуск идет нормально, результат как в предыдущем задании.
n = X.shape[1]
eta = 1e-2
w = np.array([1, 0.5])
for i in range(1001):
    #ИЗМЕНЕНО: поменяла множители местами
    y_pred = np.dot(X.T, w)
    err = calc_mse(y, y_pred)
    #ИЗМЕНЕНО: X.T заменила на X, 
    #добавила axis = 1, чтобы сложение шло только по столбцам, 
    #а изачально складывался весь массив и получался скаляр.
    w -= (eta * (1/n * 2 * np.sum(X * (y_pred - y), axis = 1)))
    if i % 100 == 0:
        print(i, w, err)

0 [2.08 4.27] 3047.75
100 [28.38281518  6.83710367] 177.42704441959268
200 [38.38986469  5.02247953] 65.32695101413637
300 [42.39314129  4.29654705] 47.386842165377494
400 [43.99463466  4.00614091] 44.51576957544466
500 [44.63530512  3.8899652 ] 44.0562931092674
600 [44.89160255  3.84348962] 43.98276009456375
700 [44.99413322  3.82489726] 43.97099212677991
800 [45.03515017  3.81745947] 43.969108822167414
900 [45.05155882  3.81448401] 43.968807424651004
1000 [45.05812303  3.8132937 ] 43.96875919004132


**Задача 3**

3(*). Вместо того, чтобы задавать количество итераций, задайте условие остановки алгоритма - когда ошибка за итерацию начинает изменяться ниже определенного порога. 

Сколько нужно сделать итераций, если установить допустимое отклонение mse в размере diff=1e-6, а значение eta=1e-2?

In [218]:
eta = 1e-2
w = np.array([1, 0.5])
epsilon = 1e-6
err_delta = np.inf
iteration = 0
err = np.inf

while epsilon <= err_delta:
    iteration += 1
    y_pred = np.dot(X.T, w)
    err_new = calc_mse(y, y_pred)
    err_delta = err - err_new  
    w -= (eta * (1/n * 2 * np.sum(X * (y_pred - y), axis = 1)))
    err = err_new 
    if iteration % 100 == 0:
        print(iteration, w, err)
        
print(f'Нужно сделать {iteration} итераций, чтобы ошибка MSE начала изменяться меньше {epsilon} скорости обучение = {eta}')
    

100 [28.22929764  6.86494171] 179.89501370640295
200 [38.32845066  5.03361602] 65.72191623045093
300 [42.36857287  4.30100215] 47.450051024147385
400 [43.98480618  4.00792316] 44.525885301240166
500 [44.63137328  3.89067818] 44.05791199482176
600 [44.89002963  3.84377484] 43.98301917537428
700 [44.99350398  3.82501136] 43.97103358917125
800 [45.03489844  3.81750512] 43.969115457664245
900 [45.05145812  3.81450228] 43.968808486572826
Нужно сделать 905 итераций, чтобы ошибка MSE начала изменяться меньше 1e-06 скорости обучение = 0.01


**Ответ по задаче 3**

Нужно сделать 905 итераций, чтобы ошибка MSE начала изменяться меньше 1e-06 скорости обучение = 0.01